In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Preprocessed.csv")

In [192]:
dataset

,Client ID,Project Size (CAD),Payment Terms (Days),Historical Defaults,Disputes Filed,Average Payment Delay (Days),Past Projects,Credit Score (0-100),Credit Risk,Industry_Industrial,Industry_Infrastructure,Industry_Residential
0,1000,0.068842,90,0,0.0,29.000,1,83,0,0,1,0
1,1001,-0.315488,90,0,0.0,25.000,6,87,0,1,0,0
2,1002,0.073534,60,0,0.0,3.000,9,87,0,0,0,0
3,1003,-1.996506,45,0,2.5,19.000,8,74,0,0,1,0
4,1004,-0.236507,30,0,2.0,12.000,11,78,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
195,1195,-1.447265,60,0,1.0,0.000,12,87,0,0,0,0
196,1196,1.140151,60,0,1.0,41.375,1,67,1,0,0,0
197,1197,-0.007629,45,0,0.0,2.000,22,77,1,1,0,0
198,1198,-0.994941,45,0,0.0,25.000,20,69,0,0,0,0


In [193]:
from sklearn.feature_selection import RFE

In [194]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [195]:
def rfe_feature(indep,dep,n):
    from sklearn.linear_model import LogisticRegression
    from sklearn.svm import SVC
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    Linearmodel=LogisticRegression(max_iter=2000)
    SVCmodel=SVC(kernel='linear')
    DTmodel=DecisionTreeClassifier(criterion='entropy',max_features=None,splitter='best',random_state=0)
    RFmodel=RandomForestClassifier(criterion='entropy',n_estimators=10,random_state=0)
    # rfemodellist=[Linearmodel,SVRmodel,DTmodel,RFmodel]

    best_col=[]
    
    # for i in rfemodellist:
    #     rfe=RFE(estimator=i,n_features_to_select=n)
    #     fit=rfe.fit(indep,dep)
    #     best_features=fit.transform(indep)
    #     best.append(best_features)
    #     col=indep.columns[fit.support_].to_list()
    #     bestcol.append(col)
    # return best,bestcol

    rfemodellist = [('Logisitc Regression',Linearmodel),('SVM', SVCmodel),('Decision Tree', DTmodel),('Random Forest', RFmodel)]
    
    feature_dict = {}

    for name, model in rfemodellist:
        rfe = RFE(estimator=model, n_features_to_select=n)
        fit = rfe.fit(indep, dep)
        best_features=fit.transform(indep)
        best_col.append(best_features)
        selected_cols = indep.columns[fit.support_].tolist()
        feature_dict[name] = selected_cols
    
    return best_col,feature_dict

In [196]:
from sklearn.model_selection import train_test_split
def TTSplit(indep,dep):
    x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)
    return x_train,x_test,y_train,y_test

In [197]:
from sklearn.metrics import accuracy_score

In [198]:
def Log(x_train,x_test,y_train,y_test):
    Classifier=LogisticRegression(max_iter=1000)
    Classifier=Classifier.fit(x_train,y_train)
    y_pred=Classifier.predict(x_test)
    acc=accuracy_score(y_test,y_pred)
    return acc

In [199]:
from sklearn.model_selection import GridSearchCV

In [200]:
def SVM(x_train,x_test,y_train,y_test):
    param_grid={'kernel':['linear','rbf'],
                'gamma':['scale','auto'],
                'C':[0.01,0.1,0.5,1.0]}
    Grid=GridSearchCV(SVC(),param_grid,scoring='f1_weighted',refit=True,cv=5,n_jobs=-1,verbose=1)
    Grid=Grid.fit(x_train,y_train)
    y_pred=Grid.predict(x_test)
    acc=accuracy_score(y_test,y_pred)
    return acc

In [201]:
def DT(x_train,x_test,y_train,y_test):
    param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'max_features': [None, 'sqrt', 'log2']}
    Grid=GridSearchCV(DecisionTreeClassifier(),param_grid,scoring='f1_weighted',refit=True,cv=5,n_jobs=-1,verbose=1)
    Grid=Grid.fit(x_train,y_train)
    y_pred=Grid.predict(x_test)
    acc=accuracy_score(y_test,y_pred)
    return acc

In [202]:
def RF(x_train,x_test,y_train,y_test):
    param_grid = {'criterion': ['gini', 'entropy', 'log_loss'],
    'n_estimators': [1, 10, 20, 50, 70, 100],
    'max_features': [None, 'sqrt', 'log2']}

    Grid=GridSearchCV(RandomForestClassifier(),param_grid,scoring='f1_weighted',refit=True,cv=5,n_jobs=-1,verbose=1)
    Grid=Grid.fit(x_train,y_train)
    y_pred=Grid.predict(x_test)
    acc=accuracy_score(y_test,y_pred)
    return acc

In [203]:
def knn(x_train,x_test,y_train,y_test):
    classifier=KNeighborsClassifier(n_neighbors=5,metric='minkowski',p=2)
    classifier=classifier.fit(x_train,y_train)
    y_pred=classifier.predict(x_test)
    acc=accuracy_score(y_test,y_pred)
    return acc

In [204]:
def naive(x_train,x_test,y_train,y_test):
    classifier=GaussianNB()
    classidfier=classifier.fit(x_train,y_train)
    y_pred=classifier.predict(x_test)
    acc=accuracy_score(y_test,y_pred)
    return acc

In [205]:
dataset

,Client ID,Project Size (CAD),Payment Terms (Days),Historical Defaults,Disputes Filed,Average Payment Delay (Days),Past Projects,Credit Score (0-100),Credit Risk,Industry_Industrial,Industry_Infrastructure,Industry_Residential
0,1000,0.068842,90,0,0.0,29.000,1,83,0,0,1,0
1,1001,-0.315488,90,0,0.0,25.000,6,87,0,1,0,0
2,1002,0.073534,60,0,0.0,3.000,9,87,0,0,0,0
3,1003,-1.996506,45,0,2.5,19.000,8,74,0,0,1,0
4,1004,-0.236507,30,0,2.0,12.000,11,78,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
195,1195,-1.447265,60,0,1.0,0.000,12,87,0,0,0,0
196,1196,1.140151,60,0,1.0,41.375,1,67,1,0,0,0
197,1197,-0.007629,45,0,0.0,2.000,22,77,1,1,0,0
198,1198,-0.994941,45,0,0.0,25.000,20,69,0,0,0,0


In [206]:
indep=dataset.drop('Credit Risk',axis=1)
dep=dataset.iloc[:,8]

In [190]:
best_col,features=rfe_feature(indep,dep,12)

D:\Anaconda3\envs\mlproject\Lib\site-packages\sklearn\feature_selection\_rfe.py:300: UserWarning: Found n_features_to_select=12 > n_features=7. There will be no feature selection and all features will be kept.
  warnings.warn(
D:\Anaconda3\envs\mlproject\Lib\site-packages\sklearn\feature_selection\_rfe.py:300: UserWarning: Found n_features_to_select=12 > n_features=7. There will be no feature selection and all features will be kept.
  warnings.warn(
D:\Anaconda3\envs\mlproject\Lib\site-packages\sklearn\feature_selection\_rfe.py:300: UserWarning: Found n_features_to_select=12 > n_features=7. There will be no feature selection and all features will be kept.
  warnings.warn(
D:\Anaconda3\envs\mlproject\Lib\site-packages\sklearn\feature_selection\_rfe.py:300: UserWarning: Found n_features_to_select=12 > n_features=7. There will be no feature selection and all features will be kept.
  warnings.warn(


In [191]:
best_col

[array([[ 6.88417509e-02,  0.00000000e+00,  2.90000000e+01, ...,
          8.30000000e+01,  0.00000000e+00,  1.00000000e+00],
        [-3.15488143e-01,  0.00000000e+00,  2.50000000e+01, ...,
          8.70000000e+01,  1.00000000e+00,  0.00000000e+00],
        [ 7.35341945e-02,  0.00000000e+00,  3.00000000e+00, ...,
          8.70000000e+01,  0.00000000e+00,  0.00000000e+00],
        ...,
        [-7.62910088e-03,  0.00000000e+00,  2.00000000e+00, ...,
          7.70000000e+01,  1.00000000e+00,  0.00000000e+00],
        [-9.94940730e-01,  0.00000000e+00,  2.50000000e+01, ...,
          6.90000000e+01,  0.00000000e+00,  0.00000000e+00],
        [ 4.42222278e-01,  2.00000000e+00,  6.00000000e+00, ...,
          7.80000000e+01,  0.00000000e+00,  1.00000000e+00]],
       shape=(200, 7)),
 array([[ 6.88417509e-02,  0.00000000e+00,  2.90000000e+01, ...,
          8.30000000e+01,  0.00000000e+00,  1.00000000e+00],
        [-3.15488143e-01,  0.00000000e+00,  2.50000000e+01, ...,
          8.700

In [152]:
features
features.items()
for model_name, col in features.items():
    print(f"{model_name}: {col}")

Logisitc Regression: ['Project Size (CAD)', 'Disputes Filed', 'Average Payment Delay (Days)', 'Past Projects', 'Credit Score (0-100)', 'Industry_Industrial', 'Industry_Infrastructure']
SVM: ['Client ID', 'Project Size (CAD)', 'Payment Terms (Days)', 'Disputes Filed', 'Average Payment Delay (Days)', 'Credit Score (0-100)', 'Industry_Infrastructure']
Decision Tree: ['Client ID', 'Project Size (CAD)', 'Payment Terms (Days)', 'Disputes Filed', 'Average Payment Delay (Days)', 'Past Projects', 'Credit Score (0-100)']
Random Forest: ['Client ID', 'Project Size (CAD)', 'Payment Terms (Days)', 'Disputes Filed', 'Average Payment Delay (Days)', 'Past Projects', 'Credit Score (0-100)']


In [207]:
acclog=[]
accsvm=[]
accdt=[]
accrf=[]
accknn=[]
accnaive=[]

for i in best_col:
    x_train,x_test,y_train,y_test=TTSplit(i,dep)

    LRscore=Log(x_train,x_test,y_train,y_test)
    acclog.append(LRscore)
    print("Logistic for Done Successfully")
    SVMscore=SVM(x_train,x_test,y_train,y_test)
    accsvm.append(SVMscore)
    print("SVM Done Successfully")
    DTscore=DT(x_train,x_test,y_train,y_test)
    accdt.append(DTscore)
    print("Decision Tree Done Successfully")
    RFscore=RF(x_train,x_test,y_train,y_test)
    accrf.append(RFscore)
    print("Random Forest Done Successfully")
    KNNscore=knn(x_train,x_test,y_train,y_test)
    accknn.append(KNNscore)
    print("KNN Done Successfully")
    naivescore=naive(x_train,x_test,y_train,y_test)
    accnaive.append(naivescore)
    print("NB Done Successfully")

D:\Anaconda3\envs\mlproject\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic for Done Successfully
Fitting 5 folds for each of 16 candidates, totalling 80 fits
SVM Done Successfully
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Decision Tree Done Successfully
Fitting 5 folds for each of 54 candidates, totalling 270 fits
Random Forest Done Successfully
KNN Done Successfully
NB Done Successfully


In [209]:
scoreboard=pd.DataFrame(index=['Linear_RFE','SVM_RFE','DT_RFE','RF_RFE'],columns=['Logistic','SVM','DT','RF','KNN','Naive'])

In [210]:
for number,index in enumerate(scoreboard.index):
    scoreboard['Logistic'][index]=acclog[number]
    scoreboard['SVM'][index]=accsvm[number]
    scoreboard['DT'][index]=accdt[number]
    scoreboard['RF'][index]=accrf[number]
    scoreboard['KNN'][index]=accknn[number]
    scoreboard['Naive'][index]=accnaive[number]

C:\Users\kathi\AppData\Local\Temp\ipykernel_11236\3992713752.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  scoreboard['Logistic'][index]=acclog[number]
C:\Users\kathi\AppData\Local\Temp\ipykernel_11236\3992713752.py:3: FutureWarning: C

In [ ]:
scoreboard

In [158]:
indep=dataset[['Project Size (CAD)', 'Disputes Filed', 'Average Payment Delay (Days)',
               'Past Projects', 'Credit Score (0-100)', 'Industry_Industrial', 'Industry_Infrastructure']]

In [160]:
indep

,Project Size (CAD),Disputes Filed,Average Payment Delay (Days),Past Projects,Credit Score (0-100),Industry_Industrial,Industry_Infrastructure
0,0.068842,0.0,29.000,1,83,0,1
1,-0.315488,0.0,25.000,6,87,1,0
2,0.073534,0.0,3.000,9,87,0,0
3,-1.996506,2.5,19.000,8,74,0,1
4,-0.236507,2.0,12.000,11,78,0,1
...,...,...,...,...,...,...,...
195,-1.447265,1.0,0.000,12,87,0,0
196,1.140151,1.0,41.375,1,67,0,0
197,-0.007629,0.0,2.000,22,77,1,0
198,-0.994941,0.0,25.000,20,69,0,0


In [161]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
195    0
196    1
197    1
198    0
199    1
Name: Credit Risk, Length: 200, dtype: int64

In [162]:
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)

In [183]:
x_test

,Project Size (CAD),Disputes Filed,Average Payment Delay (Days),Past Projects,Credit Score (0-100),Industry_Industrial,Industry_Infrastructure
18,-0.408173,0.0,6.0,10,76,1,0
170,-0.490640,0.0,14.0,5,85,1,0
107,0.462500,0.0,17.0,17,73,1,0
98,0.257638,0.0,23.0,3,84,0,0
177,1.010036,0.0,14.0,13,94,0,1
182,0.424020,0.0,15.0,3,65,0,0
5,0.337701,0.0,9.0,22,87,1,0
146,0.665378,0.0,15.0,23,59,0,1
12,-0.545210,0.0,3.0,15,73,0,1
152,-0.729925,0.0,26.0,23,79,0,0


In [164]:
classifier=GaussianNB()
classidfier=classifier.fit(x_train,y_train)
y_pred=classifier.predict(x_test)
acc=accuracy_score(y_test,y_pred)
acc

0.7666666666666667

In [165]:
import pickle

In [166]:
pickle.dump(classifier,(open("final_model.sav",'wb')))

In [184]:
# Duration=int(input("Enter estimated duration of Project(Months)="))
# InitBudg=float(input("Enter Intial Budget(CAD)="))
# LabCos=float(input("Enter estimated Labor Cost (CAD)="))
# MatCos=float(input("Enter estimated Material Cost (CAD)="))
# EqCos=float(input("Enter estimated Equipment Cost (CAD)="))
# MiscCos=float(input("Enter estimated Miscellaneous Costs (CAD)="))
# Delay=int(input("Enter delay in months="))
# ChOrders=int(input("Enter estimated number of Change Orders="))
result=classifier.predict([[-0.052373,0.0,5.0,23,76,0,1]])
# result1=classifier.predict([[0,0,0,0,0,0,0]])
# result=regressor.predict([[Duration,InitBudg,LabCos,MatCos,EqCos,MiscCos,Delay,ChOrders]])

D:\Anaconda3\envs\mlproject\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


In [185]:
result

array([0])